## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [2]:
###################################
# Load packages
###################################
from sqlalchemy import create_engine
import pandas as pd
###################################
# Create engine from local SQLite
###################################
engine=create_engine('sqlite:///E:\\ProgramData\\sqlite\\country_club.sqlite')
table_names=engine.table_names()
table_names

['Bookings', 'Facilities', 'Members']

QUESTIONS:

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [28]:
query=('SELECT name, total_revenue FROM '
           '(SELECT name, SUM(cost) AS total_revenue FROM '
               '(SELECT b.facid, f.name, b.memid, b.slots, f.membercost, f.guestcost, '
               'CASE WHEN b.memid=0 THEN slots*guestcost ELSE slots*membercost END AS cost '
               'FROM Bookings AS b '
               'LEFT JOIN Facilities AS f '
               'ON b.facid=f.facid) '
           'GROUP BY name) '
       'WHERE total_revenue<1000 '
       'ORDER BY total_revenue;'
      )
facilities_with_total_revenue_less_than_1000=pd.read_sql_query(query, engine)
facilities_with_total_revenue_less_than_1000  

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [39]:
query=("SELECT m1.surname||', '||m1.firstname AS member_name, " 
       "m2.surname||', '||m2.firstname AS recommended_by "
       'FROM Members as m1 '
       'LEFT JOIN Members as m2 ON m1.recommendedby=m2.memid '
       'ORDER BY member_name;'
)
report_of_members=pd.read_sql_query(query, engine)
report_of_members

,member_name,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


Q12: Find the facilities with their usage by member, but not guests

In [61]:
query=('WITH book AS (SELECT facid, memid '
       'FROM Bookings '
       'WHERE memid<>0) '
       "SELECT f.name AS facilities, m.surname||', '||m.firstname AS usage_by_member "
       'FROM book '
       'LEFT JOIN Facilities as f ON book.facid=f.facid '
       'LEFT JOIN Members as m ON book.memid=m.memid '
       'ORDER BY facilities, usage_by_member;'
)
facilities_with_usage_by_member=pd.read_sql_query(query, engine)
facilities_with_usage_by_member['times']=1
table=pd.pivot_table(facilities_with_usage_by_member,
                     index=['facilities'],
                     values='times',
                     columns=['usage_by_member'],
                     aggfunc='count').fillna(0).astype(int)
table.T

facilities,Badminton Court,Massage Room 1,Massage Room 2,Pool Table,Snooker Table,Squash Court,Table Tennis,Tennis Court 1,Tennis Court 2
usage_by_member,,,,,,,,,
"Bader, Florence",9,0,2,23,33,2,42,1,8
"Baker, Anne",10,3,2,12,0,49,1,6,35
"Baker, Timothy",7,24,0,85,0,5,24,14,7
"Boothe, Tim",12,36,0,25,43,12,4,4,52
"Butters, Gerald",20,32,1,6,34,9,1,57,3
"Coplin, Joan",0,1,2,11,10,1,21,7,0
"Crumpet, Erica",2,2,0,0,0,0,2,1,0
"Dare, Nancy",10,19,5,19,23,0,5,25,11
"Farrell, David",0,0,0,25,1,1,0,6,1


Q13: Find the facilities usage by month, but not guests

In [64]:
query=("WITH book AS (SELECT facid, strftime('%m', starttime) AS usage_by_month "
       'FROM Bookings '
       'WHERE memid<>0) '
       'SELECT f.name AS facilities, book.usage_by_month AS usage_by_month '
       'FROM book '
       'LEFT JOIN Facilities as f ON book.facid=f.facid '
       'ORDER BY facilities, usage_by_month;'
)
facilities_with_usage_by_month=pd.read_sql_query(query, engine)
facilities_with_usage_by_month['times']=1
table=pd.pivot_table(facilities_with_usage_by_month,
                     index=['facilities'],
                     values='times',
                     columns=['usage_by_month'],
                     aggfunc='count').astype(int)
table.T

facilities,Badminton Court,Massage Room 1,Massage Room 2,Pool Table,Snooker Table,Squash Court,Table Tennis,Tennis Court 1,Tennis Court 2
usage_by_month,,,,,,,,,
07,51,77,4,103,68,23,48,65,41
08,132,153,9,272,154,85,143,111,109
09,161,191,14,408,199,87,194,132,126
